In [ ]:
from course_settings import set_tf_nthreads
set_tf_nthreads(4) # best setting for this tutorial at CIP

# A particle physics application for CNNs: Top quark tagging

This tutorial uses a lot of material from Lisa Benato and Dirk Krücker (https://github.com/dkgithub/wuhan_DL_labs)

### The Standard Model and the top quark

<br>
<img src="figures/top_tagging/SM.png" width="400" >

The **Standard Model** of elementary particles represents our knowledge of the microscopic world. It describes the matter constituents (quarks and leptons) and their interactions (mediated by bosons), that are the electromagnetic, the weak and the strong interactions.

Among all these particles, the **top quark** still represents a very peculiar case. It is the heaviest known elementary particle (mass of 172.5 GeV) and it has a very short lifetime ($10^{-25}$ seconds): this means we can only see its decay products. It has been discovered in 1995 by the CDF and D0 experiments at Tevatron (Fermilab, Chicago). The top quark is considered a key particle to searches for new physics beyond the Standard Model and to precision measurements.

The ideal tool for measuring the top quark properties is a particle collider. The **Large Hadron Collider** (LHC), situated nearby Geneva, between France and Switzerland, is the largest proton-proton collider ever built on Earth. It consists of a 27 km circumference ring, where proton beams are smashed at a centre-of-mass energy of 13 TeV (99.999999% the speed of light). At the LHC, 40 Million collisions / second occur, yielding an enormous amount of data. Thanks to these data, **ATLAS** and **CMS** experiments discovered the missing piece of the Standard Model, the Higgs boson, in 2012.

During a collision, the energy is so high that protons are "broken" into their fundamental components, i.e. **quarks** and **gluons**, that can interact, producing particles that we don't observe in our everyday life, such as the top quark. The production of a top quark is, by the way, a relatively "rare" phenomenon, since there are other physical processes that occur way more often, such as those initiated by strong interaction, producing lighter quarks (such as up, down, strange quarks). In high energy physics, we speak about the **cross-section** of a process. We say that the top quark production has a smaller cross-section than the production of light quarks.

The experimental consequence is that distinguishing the decay products of a top quark from a light quark can be extremely difficult, given that the latter process has a way larger probability to happen.

### Experimental signature of top quark in a particle detector

Let's first understand what are the experimental signatures and how our detectors work. This is a sketch of the CMS experiment.

<br>
<img src="figures/top_tagging/EPS_CMS_Slice.png" width="1000" >

A collider detector is organized in layers: each layer is able to distinguish and measure different particles and their properties. For example, the silicon tracker detects each particle that is charged. The electromagnetic calorimeter detects photons and electrons. The hadronic calorimeter detects hadrons (such as protons and neutrons). The muon chambers detect muons (that have a long lifetime and travel through the inner layers).

Our physics problem consists into detecting the so-called "hadronic decay" of a top quark. The decay chain is sketched here: the top quark decays into a bottom quark and into a $W$ boson, that in turn decays into light quarks (in the picture, up and down quarks).

<br>
<img src="figures/top_tagging/top.png" width="500" >

Our background is, instead, represented by light quark (or quarks) produced by the strong interaction (in jargon, QCD). Here we have a sketch of one possible background event.

<br>
<img src="figures/top_tagging/QCD.png" width="200" >

#### Jets

Without going into the theoretical details, the nature of particles experiencing the strong interaction (like quarks) is such that they cannot travel free, but they are forced to be "confined" into hadrons. One hadron can be seen as a "combination" of quarks. Let's think about the electromagnetic interaction: a positive charge and a negative charge are attracted to each other, and they will tend to form a state that is neutral under the electromagnetic interaction. Analogously, quarks try to combine together, forming a bond state that is neutral under the strong interaction. This process is called **hadronization**, and it has a very important consequence. Quarks won't appear as single isolated particles in a detector, but rather as **jets** of particles.

There are many different algorithms that are able to reconstruct quarks (and gluons) as jets (i.e., anti-$k_T$ algorithm [arXiv:0802.1189](https://arxiv.org/abs/0802.1189)). They basically loop over the shower of particles produced by the hadronization, trying to cluster them together as one single entity. The algorithms are designed such in a way that the momentum of the clustered jet is proportional to the initial energy of the quark. A sketch giving an intuitive idea of a jet is displayed here (Klaus Rabbertz, KIT):

<br>
<img src="figures/top_tagging/Rabbertz_from_quark_to_rec_jet.png" width="500" >

#### Jets substructure

Many physically motivated approaches have been used in the past to distinguish a jet initiated by a top quark from jets due to QCD. One remarkable property is the so-called **jet substructure**. The idea is to try to distinguish how many "sub-jets" are included in a jet. Out of our sketches presented before, since the top quark decays into three separated quarks, we would expect it to show a three-pronged sub-structure. QCD, on the other hand, is mainly due to single quark/gluon radiation, hence it shows a one-pronged sub-structure. One largely used approach to study the jet substructure is the so called *n-subjettiness* ([arxiv:1011.2268](https://arxiv.org/abs/1011.2268)).

## The dataset

The authors of [arXiv:1707.08966](https://arxiv.org/abs/1707.08966) provide us with a dataset for studying this problem. There is also a [summary paper](https://arxiv.org/abs/1902.09914) reviewing different methods.

We downloaded the data for the course at CIP - if you want to run this exercise at home you can download the data at https://desycloud.desy.de/index.php/s/llbX3zpLhazgPJ6 (1.6 GB).

In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
# CIP pool: /large_tmp/LMU_DA_ML_19Adv/top_tagging
data_dir = "/large_tmp/LMU_DA_ML_19Adv/top_tagging"

The dataset contains about 1M training examples. For now we will just use 100k for training and testing and 10k for validation during the training.

In [ ]:
n_examples = 100000
df_train = pd.read_hdf(os.path.join(data_dir, "train.h5"), "table", stop=n_examples)
df_val = pd.read_hdf(os.path.join(data_dir, "val.h5"), "table", stop=int(n_examples / 10))
df_test = pd.read_hdf(os.path.join(data_dir, "test.h5"), "table", stop=n_examples)

The dataset contains the Lorentz Vectors $(E, p_x, p_y, p_z)$ for the leading 200 constituents of the jets. The field `is_signal_new` flags whether the jet is a QCD jet (0) or a Top quark jet (1).

In [ ]:
df_train.head()

## Jet images -  a nail for the hammer?

In the previous notebook we saw the power of convolutional neural networks when it comes to image classification. Can we transform our jet constituent data into an image? Yes we can! We unroll the cylindrical surface of the detector along the azimuthal and longitudinal coordinates and subdivide the area into pixels. The pixel values then correspond to the energy deposits (component transverse to the beam direction) of our jet constituents. Here we will use this as a grayscale image, but in principle one could use multiple features, similar to the colours of images with more information than just the energy (e.g. number of particles, energy for neutral and charged particles as done in https://arxiv.org/abs/1612.01551)

<br>
<img src="figures/top_tagging/images_jets.png" width="800" >

(Figure from https://arxiv.org/abs/1612.01551)

## Preprocessing

Since our jets are already clustered with a fixed radius parameter it is convenient to transform the coordinates such that the leading constituent is in the center of the image. As azimuthal and longitudinal coordinates we use the angle $\phi$ and the [pseudorapidity](https://en.wikipedia.org/wiki/Pseudorapidity) $\eta$, a quantity where differences are invariant under boost in beam direction. The images will be created by summing (histogramming) transverse momentum values into 40x40 pixels and normalize by the leading constituents transverse momentum (center of the image).

In [ ]:
def get_df_rel(df):
    """
    Create dataframe with PT, ETA, PHI in coordinates relative to leading constituent
    """

    # make new df with relative coordinates (to leading constituent)
    # first, just copy the labels for convenience
    df_rel = df[["is_signal_new"]].copy()

    # Augment with pt, eta, phi
    for i in range(200):
        df_rel["PT_{}".format(i)] = np.sqrt(df["PX_{}".format(i)]**2 + df["PY_{}".format(i)]**2)
        df_rel["ETA_{}".format(i)] = np.arcsinh(df["PZ_{}".format(i)]/df_rel["PT_{}".format(i)])
        df_rel["PHI_{}".format(i)] = np.arcsin(df["PY_{}".format(i)]/df_rel["PT_{}".format(i)])

    PT_0 = df_rel.PT_0.copy()
    ETA_0 = df_rel.ETA_0.copy()
    PHI_0 = df_rel.PHI_0.copy()
    for i in range(200):
        # normalize by leading constituent
        df_rel["PT_{}".format(i)] = df_rel["PT_{}".format(i)] / PT_0
        
        # shift coordinates
        df_rel["ETA_{}".format(i)] = df_rel["ETA_{}".format(i)] - ETA_0
        df_rel["PHI_{}".format(i)] = df_rel["PHI_{}".format(i)] - PHI_0

    df_rel.fillna(0, inplace=True)
    return df_rel

In [ ]:
df_rel_train = get_df_rel(df_train)

In [ ]:
df_rel_train.head()

How does an average jet image look like now?

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
def plot_avg(df, label=1):
    
    columns = sum([["PT_{}".format(i), "ETA_{}".format(i), "PHI_{}".format(i)] for i in range(200)], [])

    # transform to reshaped numpy array of particles (irrespective of event)
    trf = df[df["is_signal_new"]==label][columns].values.reshape(-1, 3)
    pt = trf[:,0]
    eta = trf[:,1]
    phi = trf[:,2]

    plt.hist2d(
        eta, phi, bins=(40, 40), range=([-1, 1], [-1, 1]),
        # the pixel intensity is the transverse momentum, so we have to weight by pt here
        weights=pt,
        norm=matplotlib.colors.LogNorm(),
    )
    plt.colorbar()
    plt.xlabel("eta")
    plt.ylabel("phi")

Average QCD jet

In [ ]:
plot_avg(df_rel_train, label=0)

Average Top quark jet

In [ ]:
plot_avg(df_rel_train, label=1)

One can see a faint "ring" around the center for top quark jets - potentially coming from the additional high momentum constituents in the top quark decay. This could be made more visible with a more sophisticated transformation, but we leave it as it is now.

For training a CNN we now have to make an array of these images:

In [ ]:
def get_img_array(df):
    """
    Pixelate constituent arrays per jet
    """
    columns = sum([["PT_{}".format(i), "ETA_{}".format(i), "PHI_{}".format(i)] for i in range(200)], [])
    hists = []
    trf = df[columns].values.reshape(-1, 200, 3)
    for i in range(len(trf)):
        pt = trf[i][:,0]
        eta = trf[i][:,1]
        phi = trf[i][:,2]
        # remember: the pixel intensity is the transverse momentum, so we have to weight by pt here
        hist, xedges, yedges = np.histogram2d(eta, phi, bins=(40, 40), range=([-1, 1], [-1, 1]), weights=pt)
        hists.append(np.array([hist]))
    return np.stack(hists).reshape(-1, 40, 40, 1)

In [ ]:
x_train = get_img_array(df_rel_train)

In [ ]:
y_train = df_rel_train.is_signal_new.values

In [ ]:
def show_random_pictures(x):
    pictures = x
    rnd_idx = np.random.permutation(len(pictures))
    fig, axs = plt.subplots(nrows=1, ncols=5, figsize=(20,4))
    for i, ax in enumerate(axs.reshape(-1)):
        picture = pictures[rnd_idx[i]]
        ax.imshow(picture.reshape(picture.shape[:-1]), cmap='inferno')

In [ ]:
show_random_pictures(x_train[y_train==0])

In [ ]:
show_random_pictures(x_train[y_train==1])

As you can see the pictures are rather sparse compared to typical images

## NN and training

Let's use the same model we had in the [previous exercise](CNNIntro.ipynb). This time we have a binary classification (top vs QCD jets) so we let the NN output a single value between 0 and 1 (similar to the exercise in [NNFromScratch.ipynb](NNFromScratchNumpy.ipynb)).

In [ ]:
import keras
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [ ]:
model = keras.models.Sequential([
    Conv2D(32, 3, activation="relu", input_shape=(40, 40, 1)),
    Conv2D(32, 3, activation="relu"),
    MaxPooling2D(),
    Dropout(0.25),
    Conv2D(64, 3, activation="relu"),
    Conv2D(64, 3, activation="relu"),
    MaxPooling2D(),
    Dropout(0.25),
    Flatten(),
    Dense(512, activation="relu"),
    Dropout(0.5),
    Dense(1, activation="sigmoid")
])

In [ ]:
model.summary()

We have a few more parameters now in the final Dense layer, since our images are a bit larger.

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [ ]:
history = model.fit(
    x_train,
    y_train,
    batch_size=64,
    shuffle=True,
    epochs=2,
    # this time we explicitely pass the validation data since we have it anyway
    validation_data=(
        # x
        get_img_array(get_df_rel(df_val)),
        # y
        df_val.is_signal_new.values
    )
)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])

## ROC curve

This classification problem is an example where the numbers of candidates for each class will be very asymmetric in the actual application (although we still train with the same number for both labels).

Depending on how the classifier is used there might be much more background from QCD jets than the actual top quark jet signal. Therefore metrics such as the accuracy are not really helpful here and instead we look at the "ROC" curve that shows the true positive rate against the false positive rate (or vice versa).

In [ ]:
x_test = get_img_array(get_df_rel(df_test))
y_test = df_test.is_signal_new.values

In [ ]:
# this time the output is just a value between 0 and 1 for each instance
scores = model.predict(x_test)

In [ ]:
opt=dict(bins=100, range=(0, 1), alpha=0.5, density=True)
plt.hist(scores[y_test==0], label="QCD", **opt)
plt.hist(scores[y_test==1], label="Top Quark", **opt)
plt.yscale("log")
plt.xlabel("Pred. Top quark probability")
plt.legend();

The following function will give numbers for true postitive rate (TPR, here the top tagging efficiency), false positive rate (FPR, here the QCD jet fake rate) and the corresponding thresholds where we would need to cut the output to get the FPR, TPR pairs.

In [ ]:
from sklearn.metrics import roc_curve

In [ ]:
fpr, tpr, thr = roc_curve(y_test, scores)

In [ ]:
plt.plot(fpr, tpr)
plt.xlabel("False positive rate (QCD jet fake rate)")
plt.ylabel("True positive rate (Top quark jet efficiency)");

Since the most interesting region might be where the rejection of Gluon jets is very high (to the very left in the plot above) it is convenient to plot the true positive rate against the inverse of the false positive rate with logarithmic y-scale:

In [ ]:
plt.plot(tpr, 1. / fpr)
plt.ylabel("QCD jet rejection")
plt.xlabel("Top quark jet efficiency")
plt.yscale("log")

We can express this either as a top quark jet efficiency at a given QCD jet fake rate, e.g. for 0.1 % QCD jet fake rate (or rejection $10^3$), we get:

In [ ]:
np.max(tpr[fpr < 0.001])

and vice versa we can compute the QCD jet rejection at a chosen top quark jet efficiency, e.g. for 30 % top-quark efficiency, we have a QCD jet rejection of:

In [ ]:
1. / np.min(fpr[tpr > 0.3])